# SVD Explained
https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/
& https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html

The Singular-Value Decomposition, or SVD for short, is a matrix decomposition method for reducing a matrix to its constituent parts in order to make certain subsequent matrix calculations simpler.

For the case of simplicity we will focus on the SVD for real-valued matrices and ignore the case for complex numbers.

The SVD allows us to discover some of the same kind of information as the eigendecomposition. However, the SVD is more generally applicable.
The SVD is used widely both in the calculation of other matrix operations, such as matrix inverse, but also as a data reduction method in machine learning. SVD can also be used in least squares linear regression, image compression, and denoising data.

In [44]:
import numpy as np
from numpy import array
from numpy import diag
from numpy import dot
from numpy import zeros
from scipy.linalg import svd       #can also use svd from numpy

In [6]:
A = np.random.rand(30, 10)

In [9]:
U, s, VT = svd(A)     # The function takes a matrix and returns the U, Sigma and V^T elements.
                      # numpy documentation: 
                           # If a has more than two dimensions, then broadcasting rules apply
                           # This means that SVD is working in “stacked” mode: it iterates over all indices of the first (a.ndim - 2)  
                           # dimensions and for each combination SVD is applied to the last two indices.
                           # If a is a matrix object (as opposed to an ndarray), then so are all the return values.

In [11]:
U.shape               # m x m matrix. columns are the left-singular vectors of A
                      # numpy documentation: 
                           # columns are the eigenvectors of A*A^T
                           # corresponding eigenvalues are s^2

(30, 30)

In [12]:
s.shape               # The Sigma diagonal matrix is returned as a vector of singular values of A. s = np.diag(true sigma matrix)

(10,)

In [13]:
VT.shape               # The V matrix is returned in a transposed form, n x n matrix 
                       # columns of V are the right singular vectors of A
                       # numpy documentation:                
                           # rows of VT are the eigenvectors of A^T*A (AKA cols of V)
                           # corresponding eigenvalues are s^2

(10, 10)

## Reconstructing A
The U, s, and V elements returned from the svd() cannot be multiplied directly.

### CASE 1) When m and n are not equal (not a square matrix)
The s vector must be converted into a diagonal matrix using the diag() function. By default, this function will create a square matrix that is n x n, relative to our original matrix. This causes a problem as the size of the matrices do not fit the rules of matrix multiplication, where the number of columns in a matrix must match the number of rows in the subsequent matrix.

We can resolve this by creating a new Sigma matrix of all zero values that is m x n (e.g. more rows) and populate the first n x n part of the matrix with the square diagonal matrix calculated via diag().

In [16]:
#let's make a smaller matrix to clearly see
A = array([[1, 2], [3, 4], [5, 6]])
print(A)

U, s, VT = svd(A)

[[1 2]
 [3 4]
 [5 6]]


In [19]:
print(U)
print(s)
print(VT)

[[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]
[9.52551809 0.51430058]
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]


In [21]:
Sigma = zeros((A.shape[0], A.shape[1]))         # create m x n Sigma matrix
print(Sigma)
Sigma[:A.shape[1], :A.shape[1]] = diag(s)       # populate Sigma with n x n diagonal matrix
print(diag(s))
B = U.dot(Sigma.dot(VT))                        # reconstruct matrix u*(sigma*vt)
print(B)

[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[9.52551809 0.        ]
 [0.         0.51430058]]
[[1. 2.]
 [3. 4.]
 [5. 6.]]


### CASE 2) When m and n are equal (square matrix)
The above complication with the Sigma diagonal only exists with the case where m and n are not equal. The diagonal matrix can be used directly when reconstructing a square matrix, as follows

In [22]:
A = array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(A)

U, s, VT = svd(A)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [23]:
Sigma = diag(s)                 # create n x n Sigma matrix
print(Sigma)
B = U.dot(Sigma.dot(VT))        # reconstruct matrix u*(sigma*vt)
print(B)

[[1.68481034e+01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.06836951e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.41842475e-16]]
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


In [ ]:
Sigma.shape

### Numpy Documentation on Reconstruction
The matrix a can be reconstructed from the decomposition with two methods. (The @ operator can be replaced by the function np.matmul for python versions below 3.5.)

#### Reconstruction based on full SVD, 2D case:

In [33]:
A = np.random.randn(9, 6) + 1j*np.random.randn(9, 6)

u, s, vt = np.linalg.svd(A, full_matrices=True)
u.shape, s.shape, vt.shape

((9, 9), (6,), (6, 6))

In [42]:
#RECON METHOD 1: (u * s[..., None, :]) @ vh
A_recon1 = np.dot(u[:, :6] * s, vt)              # (u*s)*vt, transformed u from 9x9 to 9x6 to match with s and vt

np.allclose(A, A_recon1)

True

In [38]:
u[:, :6].shape

(9, 6)

In [37]:
#RECON METHOD 2: u @ (s[..., None] * vh)                                    
smat = np.zeros((9, 6), dtype=complex)           # create m x n Sigma matrix
smat[:6, :6] = np.diag(s)                        # populate Sigma with n x n diagonal matrix

A_recon2 = np.dot(u, np.dot(smat, vt))           # u*(sigma*vt)
np.allclose(A, A_recon2)

True

In [43]:
smat.shape

(9, 6)

### Reconstruction - SVD Walkthrough from Compass
A is 150x50 (not a square) np.diag created a 50x50 sigma matrix for the reconstruction

## SVD for Dimensionality Reduction
Data with a large number of features, such as more features (columns) than observations (rows) may be reduced to a smaller subset of features that are most relevant to the prediction problem.

The result is a matrix with a lower rank that is said to approximate the original matrix.

To do this we can perform an SVD operation on the original data and select the top k largest singular values in Sigma. These columns can be selected from Sigma and the rows selected from V^T.

***** these two methods are what is used in the exercise ********

In [24]:
# define a matrix
A = array([
        [1,2,3,4,5,6,7,8,9,10],
        [11,12,13,14,15,16,17,18,19,20],
        [21,22,23,24,25,26,27,28,29,30]])
print(A)
# Singular-value decomposition
U, s, VT = svd(A)

[[ 1  2  3  4  5  6  7  8  9 10]
 [11 12 13 14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27 28 29 30]]


In [28]:
Sigma = zeros((A.shape[0], A.shape[1]))         # create m x n Sigma matrix

Sigma[:A.shape[0], :A.shape[0]] = diag(s)       # populate Sigma with n x n diagonal matrix

n_elements = 2                                  # select the number of k
Sigma = Sigma[:, :n_elements]                   # k largest cols for sigma
VT = VT[:n_elements, :]                         # k largest rows for VT

B = U.dot(Sigma.dot(VT))                        # reconstruct
print(B)

T = U.dot(Sigma)                                # transform method 1 (U*Sigma)
print(T)
T = A.dot(VT.T)                                 # transform method 2 (A*V)      .T is a transpose function
print(T)

[[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [11. 12. 13. 14. 15. 16. 17. 18. 19. 20.]
 [21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]]
[[-18.52157747  -6.47697214]
 [-49.81310011  -1.91182038]
 [-81.10462276   2.65333138]]
[[-18.52157747  -6.47697214]
 [-49.81310011  -1.91182038]
 [-81.10462276   2.65333138]]
